In [84]:
import os, math, random, time

import numpy as np
import pandas as pd
import statsmodels.api as sm
import tomllib

from copy import copy
from tqdm import tqdm
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import roc_auc_score

from pathlib import Path

### Setup

In [85]:
with open("../config/mi.toml", "rb") as f:
    config = tomllib.load(f)

run_scenarios = config["run_scenarios"]
run_pymice = config["run_pymice"]
run_simice = config["run_simice"]
run_real_setup = config["run_real_setup"]

pymice_runs_count = config["pymice_runs_count"]
simice_runs_count = config["simice_runs_count"]
real_setup_runs_count = config["real_setup_runs_count"]

mi_factor = config["mi_factor"]
miss_val = config["miss_val"]
dump_data = config["dump_data"]
labels_noise_scale = config["im_noise_scale"]

paper_scenario_rows = config["paper_scenario_rows"]
midn_scenario_rows = config["midn_scenario_rows"]
midn_scenario_cols = config["midn_scenario_cols"]
midn_miss_rows_count = config["midn_miss_rows_count"]

scenarios_runs_count = config["scenarios_runs_count"]
scenarios_miss_col = config["scenarios_miss_col"]

real_setup_outlier_threshold = config["real_setup_outlier_threshold"]

random.seed(0)
np.random.seed(0)

### Library

In [86]:
""" Data generators/parsers """

def export_data(name, tampered_data, non_tampered_labels, non_tampered_data, miss_rows, miss_val):
    path = f"../../data/mi/{name}/{len(tampered_data)}"
    Path(path).mkdir(parents=True, exist_ok=True)
    
    np.savetxt(f"{path}/tampered_data.txt", np.nan_to_num(tampered_data, nan=miss_val))
    np.savetxt(f"{path}/tampered_data_w_nans.txt", tampered_data)
    np.savetxt(f"{path}/non_tampered_data.txt", non_tampered_data)
    np.savetxt(f"{path}/non_tampered_labels.txt", non_tampered_labels)
    np.savetxt(f"{path}/miss_rows.txt", miss_rows, fmt='%d')

def import_data(path):
    tampered_data = np.loadtxt(f"{path}/tampered_data_w_nans.txt")
    non_tampered_data = np.loadtxt(f"{path}/non_tampered_data.txt")
    non_tampered_labels = np.expand_dims(np.loadtxt(f"{path}/non_tampered_labels.txt"), axis=1)
    miss_rows = np.loadtxt(f"{path}/miss_rows.txt").astype(int).tolist()
    
    return tampered_data, non_tampered_labels, non_tampered_data, miss_rows

def batch(data, batch_size: int = 0, batch_count: int = 0):
    assert bool(batch_size) ^ bool(batch_count), "Invalid batching parameters: either size or count should be passed"

    data_size = data.shape[0]
    if batch_size == 0:
        batch_size = data_size // batch_count
    elif batch_count == 0:
        batch_count = (data_size + batch_size - 1) // batch_size
    
    return [data[i * batch_size:(i + 1) * batch_size] for i in range(batch_count)]

def relative_noise(arr, scale):
    if scale == 0.0:
        return np.zeros_like(arr)
    
    return np.random.normal(loc=0.0, scale=np.abs(arr) * scale)

def load_or_generate_paper_data(rows: int, scenario: str, miss_col: int, labels_noise_scale: float):
    # Generate data as in https://www.nature.com/articles/s41467-020-19270-2 scenarios
    assert scenario in {"scenario_1", "scenario_2"}, f"Invalid scenario for MI data {scenario}"

    feature_count = 2
    scenario_name = f"paper_{scenario}"
    path = f"../../data/mi/{scenario_name}/{rows}"
    if os.path.isdir(path):
        return scenario_name, *import_data(path)

    # x_2 is from U(-3, 3)
    x_2 = np.random.rand(rows) * 6 - 3
    # Normalize to avoid MPC overflow later on
    if rows > 500:
        x_2 /= float(rows // 500)

    if scenario == "linear":
        # Linear x_1 is from normal(mu=0.2 - 0.5*x_2, sigma=1.0)
        x_1 = np.random.normal(loc=0.2 - 0.5 * x_2, scale=1.0)
    else:
        # Logistic x_1 is from bernoulli(n=1, p=1 / (1 + exp(-0.2 + 0.5*x_2)))
        x_1 = np.random.binomial(n=1, p=1 / (1 + np.exp(-0.2 + 0.5 * x_2)))

    non_tampered_data = np.hstack((np.expand_dims(x_1, axis=1), np.expand_dims(x_2, axis=1)))
    
    miss_rows = []
    tampered_data = non_tampered_data.copy()
    
    non_tampered_labels = non_tampered_data @ np.ones((feature_count, 1)) + 1.0  # Weights and bias are ones
    non_tampered_labels += relative_noise(non_tampered_labels, scale=labels_noise_scale)
    for i in range(rows):
        # The variable is missing with probability 1 / (1 + math.exp(-0.3 + 0.2 * labels[i][0] - 0.1 * x_2[i]))
        if (1 / (1 + math.exp(-0.3 + 0.2 * non_tampered_labels[i][0] - 0.1 * x_2[i]))) > random.random():
            tampered_data[i][miss_col] = np.nan
            miss_rows.append(i)

    print(f"Generated data missing rate: {len(miss_rows)}/{len(non_tampered_data)}")

    scenario_tuple = (scenario_name, tampered_data, non_tampered_labels, non_tampered_data, miss_rows)
    if dump_data:
        export_data(*scenario_tuple, miss_val)

    return scenario_tuple

def load_real_data(outlier_threshold):
    # Real data
    df = pd.read_csv("../../data/mi/real/gcasr.csv")
    binary_columns = ["RaceAA", "RaceW", "HlthInsM", "HlthInsN", "Day", "NPO", "MedHisST", "MedHisTI", "MedHisVP", "MedHisFHSTK"]
    continuous_columns = ["NIHStrkS", "EMSNote", "LipTotal", "Age", "Gender"]
    col_types = ["lin" if i < len(continuous_columns) else "log" for i in range(len(binary_columns) + len(continuous_columns))]
    df_of_interest = df[continuous_columns + binary_columns]

    # Normalize continuous columns
    df_of_interest[continuous_columns] = (df_of_interest[continuous_columns] - df_of_interest[continuous_columns].mean()) / df_of_interest[continuous_columns].std()

    # Fill the missing data for numerical feasibility
    real_data = df_of_interest.to_numpy()
    # Drop the missing data to get the training data
    real_complete_data = df_of_interest.dropna()

    if dump_data:
        np.savetxt(f"../../data/mi/real/incomplete_data.txt", np.nan_to_num(real_data, nan=miss_val), fmt='%10.2f')
        np.savetxt(f"../../data/mi/real/complete_data.txt", real_complete_data.to_numpy(), fmt='%10.2f')

    non_miss_rows = real_complete_data.index.to_numpy()

    # Get labels
    x = df["x"].fillna(0)
    # Reduce the outliers
    x[x < -outlier_threshold] = -outlier_threshold
    x[x > outlier_threshold] = outlier_threshold
    # Normalize
    x = (x - x.mean()) / x.std()
    real_labels = x.to_numpy()

    if dump_data:
        np.savetxt(f"../../data/mi/real/labels.txt", real_labels, fmt='%10.2f')

    # Get missing rows
    real_miss_rows = df_of_interest.isna().to_numpy().astype(int).T

    if dump_data:
        np.savetxt(f"../../data/mi/real/miss_rows.txt", real_miss_rows, fmt='%d')

    return real_data, real_complete_data.to_numpy(), real_labels, real_miss_rows, col_types, non_miss_rows

def load_or_generate_midn_data(rows, cols, scenario, miss_rows, miss_col, labels_noise_scale):
    assert scenario in {"scenario_1", "scenario_2"}, f"Invalid scenario for MI data {scenario}"
    
    scenario_name = f"midn_{scenario}"
    path = f"../../data/mi/{scenario_name}/{rows}"
    if os.path.isdir(path):
        return scenario_name, *import_data(path)

    non_tampered_data = np.random.normal(size=(rows, cols))

    if "_2" or "_4" in scenario:
        non_tampered_data[:, 0] = np.random.binomial(1, 0.5, rows)

    non_tampered_labels = non_tampered_data @ np.ones((cols, 1)) + 1.0  # Weights and bias are ones
    non_tampered_labels += relative_noise(non_tampered_labels, scale=labels_noise_scale)

    tampered_data = non_tampered_data.copy()
    for i in range(miss_rows):
        tampered_data[i][miss_col] = np.nan
    
    scenario_tuple = (scenario_name, tampered_data, non_tampered_labels, non_tampered_data, list(range(miss_rows)))
    if dump_data:
        export_data(*scenario_tuple, miss_val)
    
    return scenario_tuple

In [87]:
""" Imputers """

class Imputer:
    def __init__(self, model):
        self.model = model
    
    @staticmethod
    def split_train_test(data, target_col: int, miss_rows):
        m, n = data.shape
        miss_count = len(miss_rows)

        complete_rows = np.zeros((m - miss_count, n - 1))
        complete_labels = np.zeros((m - miss_count,))
        incomplete_rows = np.zeros((miss_count, n - 1))

        x_idx = 0
        x_com_idx = 0
        for i in range(m):
            row = data[i]
            row_complement = np.delete(row, target_col)
            if i in miss_rows:
                incomplete_rows[x_com_idx] = row_complement
                x_com_idx += 1
            else:
                complete_rows[x_idx] = row_complement
                complete_labels[x_idx] = row[target_col]
                x_idx += 1
        
        return complete_rows, incomplete_rows, complete_labels
    
    def fit(self, complete_data, complete_labels):
        self.model.fit(X=complete_data, y=complete_labels)
    
    def predict(self, data):
        return self.model.predict_proba(data) if isinstance(self.model, LogisticRegression) else self.model.predict(data)
    
    def impute(self, data, miss_rows, miss_col: int, noise: bool):
        complete_data, incomplete_data, complete_labels = Imputer.split_train_test(data, miss_col, miss_rows)
        
        if len(incomplete_data) == 0:
            return complete_data.copy()
        
        self.model.fit(X=complete_data, y=complete_labels)
        imputed_y = self.predict(incomplete_data)
        if noise:
            imputed_y += np.random.normal(0.0, 0.01, size=imputed_y.shape)

        idx = 0
        imputed_data = data.copy()
        for i in miss_rows:
            imputed_data[i, miss_col] = imputed_y.ravel()[idx]
            idx += 1

        return imputed_data
    
    def impute(self, data, incomplete_data, miss_rows, miss_col, noise):
        imputed_y = self.predict(incomplete_data)
        if noise:
            imputed_y += np.random.normal(0.0, 0.01, size=imputed_y.shape)

        idx = 0
        imputed_data = data.copy()
        for i in miss_rows:
            imputed_data[i, miss_col] = imputed_y.ravel()[idx]
            idx += 1

        return imputed_data
    
    def impute_inplace(self, data, mask, miss_col, noise):
        if mask.sum() == 0:  # Nothing to impute --- no missing data
            return
        
        reference_data = np.hstack((data[:, :miss_col], data[:, miss_col+1:]))
        imputed_y = self.predict(reference_data[np.where(mask == 1)])
        if noise and isinstance(self.model, LinearRegression):
            imputed_y += np.random.normal(0.0, 0.01, size=imputed_y.shape)

        idx = 0
        for i in range(len(mask)):
            if mask[i]:
                data[i, miss_col] = imputed_y.ravel()[idx]
                idx += 1
    

class MI:
    def __init__(self, factor: int, impute_model, fit_model):
        self.imputer = Imputer(impute_model)
        self.model = fit_model
        self.factor = factor

    @staticmethod
    def rubin(weights):
        return np.mean(weights, axis=0)
    
    def fit(self, data, labels, miss_rows, miss_col, noise):
        complete_data, incomplete_data, complete_labels = Imputer.split_train_test(data, miss_col, miss_rows)
        
        imputed_data = []
        self.imputer.fit(complete_data=complete_data, complete_labels=complete_labels)
        
        imputed_data = [self.imputer.impute(
            data=data, incomplete_data=incomplete_data,
            miss_rows=miss_rows, miss_col=miss_col, noise=noise) for _ in range(self.factor)]
        
        self.fit_analysis_model(imputed_data, labels)
        return self, imputed_data
    
    def fit_analysis_model(self, imputed_data, labels):
        weights = np.zeros((self.factor, imputed_data[0].shape[1]))
        for i, data in enumerate(imputed_data):
            self.model.fit(X=data, y=labels)
            weights[i] = self.model.coef_.copy()
        
        self.model.coef_ = MI.rubin(weights)
    

class BatchMI:
    def __init__(self, factor: int, impute_models, fit_model):
        self.imputers = [Imputer(impute_model) for impute_model in impute_models]
        self.model = fit_model
        self.factor = factor

    def fit(self, data, complete_data, labels, miss_rows):
        miss_col = 0
        for imputer in self.imputers:
            train_data = np.hstack((complete_data[:, :miss_col], complete_data[:, miss_col+1:]))
            train_labels = complete_data[:, miss_col:miss_col+1]
            
            imputer.fit(complete_data=train_data, complete_labels=train_labels.ravel())
            miss_col += 1
        
        imputed_data = []
        weights = np.zeros((self.factor, data.shape[1]))
        for i in range(self.factor):
            inter_imputed_data = data.copy()
            self.impute_inplace(inter_imputed_data, miss_rows)
            imputed_data.append(inter_imputed_data)
            self.model.fit(X=inter_imputed_data, y=labels)
            weights[i] = self.model.coef_.copy()
        
        self.model.coef_ = MI.rubin(weights)
        return self, imputed_data
    
    def impute_inplace(self, incomplete_data, miss_rows):
        miss_col = 0
        for imputer in self.imputers:
            imputer.impute_inplace(
                data=incomplete_data,
                mask=miss_rows[miss_col], miss_col=miss_col, noise=True)
            miss_col += 1

In [88]:
""" Closed-form linear regression w.r.t. least squares"""

class ClosedFormLinReg:
    def fit(self, X, y):
        X_tilde = np.hstack((X, np.ones((X.shape[0], 1))))
        coef = np.linalg.inv(X_tilde.T @ X_tilde) @ X_tilde.T @ y
        self.coef_ = coef[:-1].flatten()
        self.intercept = coef[-1]

    def predict(self, X):
        return X @ np.expand_dims(self.coef_, axis=1) + self.intercept

In [89]:
""" Custom logistic regression """

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class CustomLogReg:
    def __init__(self, optimizer: str = "bgd"):
        self.optimizer = optimizer
    
    def fit(self, X, y, step: float = 1 / (1 << 7), epochs: int = 10, verbose: bool = False):
        self.coef_ = CustomLogReg._fit(X, y, np.random.rand(X.shape[1] + 1, 1), step, epochs, self.optimizer, verbose)
        return self
    
    def predict(self, X):
        X_tilde = np.hstack((X, np.ones((X.shape[0], 1))))
        return np.round(sigmoid((X_tilde @ self.coef_)))
    
    @staticmethod
    def loss(X, y, w):
        return CustomLogReg.log_loss(X, y, w)

    def _fit(X, y, initial_w, step: float, epochs: int, optimizer: str, verbose: bool):
        # Adding bias
        X_tilde = np.hstack((X, np.ones((X.shape[0], 1))))

        # Gradient descent
        if optimizer == "bgd":
            return CustomLogReg._bgd(X_tilde, y, initial_w, step, epochs, verbose)
        if optimizer == "mbgd":
            return CustomLogReg._mbgd(X_tilde, y, initial_w, step, epochs, 10, verbose)
        else:
            raise ValueError(f"CustomLogReg: invalid optimizer passed: {optimizer}")
    
    def _bgd(X_tilde, y, initial_w, step: float, epochs: int, verbose: bool):
        if verbose:
            print(f"Log. reg. BGD step size:", step)
        
        # Batched gradient descent
        w = initial_w  # n x 1
        y_mat = np.expand_dims(y.flatten(), axis=1)
        for _ in range(epochs):
            if verbose:
                print(f"Log. reg. BGD epoch {_ + 1}/{epochs}")
                print(f"\t weigts avg {np.mean(w)} | loss: {CustomLogReg.loss(X_tilde, y, w)}")
            
            dot = X_tilde @ w  # m x 1
            sig = sigmoid(dot)  # m x 1
            w -= X_tilde.T @ (sig - y_mat) * step  # n x 1
        
        return w
    
    def _mbgd(X_tilde, y, initial_w, step: float, epochs: int, batches: int, verbose: bool):
        if verbose:
            print(f"Log. reg. BGD step size:", step)
        
        # Compute mini-batches
        X_mini_batches = batch(X_tilde, batch_count=batches)
        y_mini_batches = batch(np.expand_dims(y.flatten(), axis=1), batch_count=batches)
        
        # Mini-batched gradient descent
        w = initial_w
        for _ in range(epochs):
            for i in range(batches):
                if verbose:
                    print(f"Log. reg. MBGD epoch {_ + 1}/{epochs} -- batch {i + 1}/{batches}")
                    print(f"\t weigts avg {np.mean(w)} | loss: {CustomLogReg.loss(X_mini_batches[i], y_mini_batches[i], w)}")
                
                dot = X_mini_batches[i] @ w
                sig = sigmoid(dot)  # m x 1
                w -= X_mini_batches[i].T @ (sig - y_mini_batches[i]) * step
        
        return w
    
    def log_loss(X, y, w):
        a = sigmoid(X @ w)
        return np.sum(-y * np.log(a) - (1 - y) * np.log(1 - a))


In [90]:
""" PyMICE """

def pymice(incomplete_data, round = False):
    imp_mean = IterativeImputer()  # set sample_posterior=True, random_state=random.randint(0, 1000) as attributes for MI
    imp_mean.fit(incomplete_data)
    imputed_data = imp_mean.transform(incomplete_data)

    if round:
        imputed_data[np.isnan(incomplete_data)] = np.round(imputed_data[np.isnan(incomplete_data)])
    
    return imputed_data

In [91]:
""" Evaluators """

def sign(x):
    return -1 if x < 0 else 1 if x > 0 else 0

def get_stat_sig(X, y):
    X2 = sm.add_constant(X)
    est = sm.OLS(np.expand_dims(y, axis=1), X2)
    est2 = est.fit()
    return est2.params, est2.pvalues

def discrepancies(data_instance, mdni_stats, sequre_stats, verbose=False):
    mdni_coeff, mdni_pval = mdni_stats
    sequre_coeff, sequre_pval = sequre_stats

    discr = 0
    for i in range(len(mdni_coeff)):
        if mdni_pval[i] <= 0.05 and (sequre_pval[i] > 0.05 or sign(mdni_coeff[i]) != sign(sequre_coeff[i])):
            if verbose:
                print(f"Discrepancy found at var {i} and instance {data_instance}! SIMICE p_val: {mdni_pval[i]}, Sequre p_val: {sequre_pval[i]}, SIMICE coeff: {mdni_coeff[i]}, Sequre coeff: {sequre_coeff[i]}")
            discr += 1
        elif verbose:
            print(f"No discrepancy found at var {i} and instance {data_instance}:\n\t\tSIMICE p_val: {mdni_pval[i]}\n\t\tSequre p_val: {sequre_pval[i]}\n\t\tSIMICE coeff: {mdni_coeff[i]}\n\t\tSequre coeff: {sequre_coeff[i]}")
    
    return discr

def count_discrepancies(imputed_datasets, mice_imputed_data, labels):
    mice_stats = get_stat_sig(mice_imputed_data, labels)

    discr = 0
    for i, imputed_dataset in enumerate(imputed_datasets):
        sequre_stats = get_stat_sig(imputed_dataset, labels)
        discr += discrepancies(i, mice_stats, sequre_stats)
    
    return discr / len(imputed_datasets)

def evaluate_scenario(scenario, impute_model, mi_factor, miss_col, miss_val, dump_data):
    (msg, tampered_data, labels, non_tampered_data, miss_rows) = scenario

    mice_imputed_data = pymice(tampered_data, isinstance(impute_model, LogisticRegression))
    if dump_data:
        np.savetxt(f"../../data/mi/{msg}/{len(tampered_data)}/py_mice_data.txt", mice_imputed_data)
    
    fit_model = LinearRegression()
    mi, imputed_datasets = MI(mi_factor, impute_model, fit_model).fit(
        np.nan_to_num(tampered_data, nan=miss_val), labels, miss_rows, miss_col, noise=True if msg.endswith("_1") else False)
    
    predicted_data = mi.model.predict(non_tampered_data).flatten()
    labels = labels.flatten()
    assert predicted_data.shape == labels.shape

    abs_diff = np.abs(predicted_data - labels)
    y_hat_y_mean = np.mean(abs_diff)
    y_hat_y_std = np.std(abs_diff)
    discrepancies_count = count_discrepancies(imputed_datasets, mice_imputed_data, labels)

    imputed_cols = [imputed_dataset[miss_rows, miss_col] for imputed_dataset in imputed_datasets]
    imputed_cols_prob = copy(imputed_cols)
    if "_2" in msg:
        imputed_cols = [np.round(imputed_col) for imputed_col in imputed_cols]
    expected_imputation = non_tampered_data[miss_rows, miss_col]
    imputation_abs_diff = sum([np.abs(imputed_col - expected_imputation) for imputed_col in imputed_cols]) / len(imputed_cols)
    imputation_diff_mean = np.mean(imputation_abs_diff)
    imputation_diff_std = np.std(imputation_abs_diff)
    imputation_accuracy = sum([(np.sum(imputed_col == expected_imputation) / len(expected_imputation)) for imputed_col in imputed_cols]) / len(imputed_cols)
    imputation_auc = sum([roc_auc_score(expected_imputation, imputed_col_prob) for imputed_col_prob in imputed_cols_prob]) / len(imputed_cols_prob)

    return mi.model.coef_, y_hat_y_mean, y_hat_y_std, discrepancies_count, imputation_diff_mean, imputation_diff_std, imputation_accuracy, imputation_auc

def benchmark_scenario(runs_count, scenario, impute_model, mi_factor, miss_col, miss_val, dump_data):
    features_count = scenario[1].shape[1]
    
    coefs = np.zeros((runs_count, features_count))
    y_hat_y_means = np.zeros((runs_count,))
    y_hat_y_sds = np.zeros((runs_count,))
    discrs = np.zeros((runs_count,))
    imp_means = np.zeros((runs_count,))
    imp_sds = np.zeros((runs_count,))
    imp_accs = np.zeros((runs_count,))
    imp_aucs = np.zeros((runs_count,))
    runtimes = np.zeros((runs_count,))
    
    for i in tqdm(range(runs_count), f"Evaluating {scenario[0]}"):
        s = time.time()
        coef, y_hat_y_mean, y_hat_y_sd, discr, imp_mean, imp_sd, imp_acc, imp_auc = evaluate_scenario(
            scenario, impute_model, mi_factor, miss_col, miss_val, dump_data)
        e = time.time()
        
        coefs[i] = coef
        y_hat_y_means[i] = y_hat_y_mean
        y_hat_y_sds[i] = y_hat_y_sd
        discrs[i] = discr
        imp_means[i] = imp_mean
        imp_sds[i] = imp_sd
        imp_accs[i] = imp_acc
        imp_aucs[i] = imp_auc
        runtimes[i] = e - s
    
    return scenario[0], coefs, y_hat_y_means, y_hat_y_sds, discrs, imp_means, imp_sds, imp_accs, imp_aucs, runtimes

def evaluate_real_setup(real_data, mi_factor, miss_val, dump_data):
    incomplete_data, complete_data, labels, miss_rows, col_types, non_miss_rows = real_data

    impute_models = [LogisticRegression() if t == "log" else LinearRegression() for t in col_types]
    fit_model = LinearRegression()
    
    mice_imputed_data = pymice(incomplete_data)
    if dump_data:
        np.savetxt(f"../../data/mi/real/py_mice_data.txt", mice_imputed_data, fmt='%10.2f')

    incompletes_w_miss_val = np.nan_to_num(incomplete_data, nan=miss_val)
    mi, imputed_datasets = BatchMI(mi_factor, impute_models, fit_model).fit(
        incompletes_w_miss_val, complete_data, labels, miss_rows)
    
    predicted_data = mi.model.predict(complete_data).flatten()
    complete_labels = labels.flatten()[non_miss_rows]
    assert predicted_data.shape == complete_labels.shape

    abs_diff = np.abs(predicted_data - complete_labels)
    y_hat_y_mean = np.mean(abs_diff)
    y_hat_y_sd = np.std(abs_diff)
    discrepancies_count = count_discrepancies(imputed_datasets, mice_imputed_data, labels.flatten())

    return mi.model.coef_, y_hat_y_mean, y_hat_y_sd, discrepancies_count

def benchmark_real_setup(runs_count, real_data, mi_factor, miss_val, dump_data):
    y_hat_y_means = np.zeros((runs_count,))
    y_hat_y_sds = np.zeros((runs_count,))
    discrs = np.zeros((runs_count,))
    runtimes = np.zeros((runs_count,))
    
    for i in tqdm(range(runs_count), "Evaluating real setup"):
        s = time.time()
        _, y_hat_y_mean, y_hat_y_sd, discr = evaluate_real_setup(real_data, mi_factor, miss_val, dump_data)
        e = time.time()

        y_hat_y_means[i] = y_hat_y_mean
        y_hat_y_sds[i] = y_hat_y_sd
        discrs[i] = discr
        runtimes[i] = e - s
    
    return None, y_hat_y_means, y_hat_y_sds, discrs, None, None, None, None, runtimes
    
def evaluate_pymice(data, labels, non_tampered_data, mi_factor, non_miss_rows = None, binary = False):
    fit_model = LinearRegression()
    imputed_datasets = [pymice(data) for _ in range(mi_factor)]

    mi = MI(mi_factor, None, fit_model)
    mi.fit_analysis_model(imputed_datasets, labels)

    if binary:
        for imputed_dataset in imputed_datasets:
            imputed_dataset[np.isnan(data)] = np.round(imputed_dataset[np.isnan(data)])
    
    predicted_data = mi.model.predict(non_tampered_data).flatten()
    labels = labels.flatten()
    if non_miss_rows is not None:
        labels = labels[non_miss_rows]
    assert predicted_data.shape == labels.shape

    abs_diff = np.abs(predicted_data - labels)
    y_hat_y_mean = np.mean(abs_diff)
    y_hat_y_sds = np.std(abs_diff)

    if non_miss_rows is None:
        imputed_cols = [imputed_dataset[np.where(np.isnan(data))] for imputed_dataset in imputed_datasets]
        imputed_cols_prob = copy(imputed_cols)
        if binary:
            imputed_cols = [np.round(imputed_col) for imputed_col in imputed_cols]
        expected_imputation = non_tampered_data[np.where(np.isnan(data))]
        imputation_abs_diff = sum([np.abs(imputed_col - expected_imputation) for imputed_col in imputed_cols]) / len(imputed_cols)
        imputation_diff_mean = np.mean(imputation_abs_diff)
        imputation_diff_std = np.std(imputation_abs_diff)
        imputation_accuracy = sum([(np.sum(imputed_col == expected_imputation) / len(expected_imputation)) for imputed_col in imputed_cols]) / len(imputed_cols)
        imputation_auc = sum([roc_auc_score(expected_imputation, imputed_col_prob) for imputed_col_prob in imputed_cols_prob]) / len(imputed_cols_prob)

        return mi.model.coef_, y_hat_y_mean, y_hat_y_sds, imputation_diff_mean, imputation_diff_std, imputation_accuracy, imputation_auc
    
    return mi.model.coef_, y_hat_y_mean, y_hat_y_sds, None, None, None

def benchmark_pymice(runs_count, msg, data, labels, non_tampered_data, mi_factor, non_miss_rows = None, binary = False):
    coefs = np.zeros((runs_count, data.shape[1]))
    y_hat_y_means = np.zeros((runs_count,))
    y_hat_y_sds = np.zeros((runs_count,))
    discrs = np.zeros((runs_count,)) - 1
    imp_means = np.zeros((runs_count,))
    imp_sds = np.zeros((runs_count,))
    imp_accs = np.zeros((runs_count,))
    imp_aucs = np.zeros((runs_count,))
    runtimes = np.zeros((runs_count,))
    
    for i in tqdm(range(runs_count), f"Evaluating pymice for {msg}"):
        s = time.time()
        coef, y_hat_y_mean, y_hat_y_sd, imp_mean, imp_sd, imp_acc, imp_auc = evaluate_pymice(data, labels, non_tampered_data, mi_factor, non_miss_rows, binary)
        e = time.time()
        
        coefs[i] = coef
        y_hat_y_means[i] = y_hat_y_mean
        y_hat_y_sds[i] = y_hat_y_sd
        imp_means[i] = imp_mean
        imp_sds[i] = imp_sd
        imp_accs[i] = imp_acc
        imp_aucs[i] = imp_auc
        runtimes[i] = e - s
    
    return msg, coefs, y_hat_y_means, y_hat_y_sds, discrs, imp_means, imp_sds, imp_accs, imp_aucs, runtimes

def evaluate_simice(scenario, tampered_data, labels, non_tampered_data, mi_factor, dump_data):
    rows = tampered_data.shape[0]
    
    fit_model = LinearRegression()
    imputed_datasets = [np.loadtxt(f"../../data/mi/siMICE/{scenario}/{rows}/imputed_data_{i + 1}.txt") for i in range(mi_factor)]
    mice_imputed_data = pymice(tampered_data, "_2" in scenario)
    if dump_data:
        np.savetxt(f"../../data/mi/siMICE/{scenario}/{rows}/py_mice_data.txt", mice_imputed_data, fmt='%10.2f')
    
    mi = MI(mi_factor, None, fit_model)
    mi.fit_analysis_model(imputed_datasets, labels)
    
    predicted_data = mi.model.predict(non_tampered_data).flatten()
    labels = labels.flatten()
    assert predicted_data.shape == labels.shape
    
    abs_diff = np.abs(predicted_data - labels)
    y_hat_y_mean = np.mean(abs_diff)
    y_hat_y_sds = np.std(abs_diff)
    discrepancies_count = count_discrepancies(imputed_datasets, mice_imputed_data, labels)

    imputed_cols = [imputed_dataset[np.where(np.isnan(tampered_data))] for imputed_dataset in imputed_datasets]
    imputed_cols_prob = copy(imputed_cols)
    if "_2" in scenario:
        imputed_cols = [np.round(imputed_col) for imputed_col in imputed_cols]
    expected_imputation = non_tampered_data[np.where(np.isnan(tampered_data))]
    imputation_abs_diff = sum([np.abs(imputed_col - expected_imputation) for imputed_col in imputed_cols]) / len(imputed_cols)
    imputation_diff_mean = np.mean(imputation_abs_diff)
    imputation_diff_std = np.std(imputation_abs_diff)
    imputation_accuracy = sum([(np.sum(imputed_col == expected_imputation) / len(expected_imputation)) for imputed_col in imputed_cols]) / len(imputed_cols)
    imputation_auc = sum([roc_auc_score(expected_imputation, imputed_col_prob) for imputed_col_prob in imputed_cols_prob]) / len(imputed_cols_prob)

    return mi.model.coef_, y_hat_y_mean, y_hat_y_sds, discrepancies_count, imputation_diff_mean, imputation_diff_std, imputation_accuracy, imputation_auc

def benchmark_simice(runs_count, scenario, tampered_data, labels, non_tampered_data, mi_factor, dump_data):
    cols = tampered_data.shape[1]
    coefs = np.zeros((runs_count, cols))
    y_hat_y_means = np.zeros((runs_count,))
    y_hat_y_sds = np.zeros((runs_count,))
    discrs = np.zeros((runs_count,))
    imp_means = np.zeros((runs_count,))
    imp_sds = np.zeros((runs_count,))
    imp_accs = np.zeros((runs_count,))
    imp_aucs = np.zeros((runs_count,))
    runtimes = np.zeros((runs_count,))
    
    for i in tqdm(range(runs_count), "Evaluating siMICE"):
        s = time.time()
        coef, y_hat_y_mean, y_hat_y_sd, discr, imp_mean, imp_sd, imp_acc, imp_auc = evaluate_simice(scenario, tampered_data, labels, non_tampered_data, mi_factor, dump_data)
        e = time.time()

        coefs[i] = coef
        y_hat_y_means[i] = y_hat_y_mean
        y_hat_y_sds[i] = y_hat_y_sd
        discrs[i] = discr
        imp_means[i] = imp_mean
        imp_sds[i] = imp_sd
        imp_accs[i] = imp_acc
        imp_aucs[i] = imp_auc
        runtimes[i] = e - s
    
    return coefs, y_hat_y_means, y_hat_y_sds, discrs, imp_means, imp_sds, imp_accs, imp_aucs, runtimes

def print_stats(msg, coefs, biases, sds, discrepancies_count, imp_biases, imp_sds, imp_accs, imp_aucs, runtimes):
    csv_out_path = "temp_mi_stats_python.csv"
    
    with open(csv_out_path, "a+") as f:
        if msg.lower().startswith("midn") or msg.lower().startswith("paper"):
            coefs_truth = np.ones_like(coefs[0])
            coefs_bias = np.linalg.norm(np.mean(coefs, axis=0) - coefs_truth)
            coefs_sd = np.sqrt(np.mean(np.sum((coefs - np.mean(coefs, axis=0)) ** 2, axis=1)))
            coefs_rmse = np.sqrt(np.mean(np.sum((coefs - coefs_truth) ** 2, axis=1)))

            print(f"{msg} | Python MI coefs bias: {coefs_bias}")
            print(f"{msg} | Python MI coefs SD: {coefs_sd}")
            print(f"{msg} | Python MI coefs rMSE: {coefs_rmse}")

            f.write(f"{coefs_bias},{coefs_sd},{coefs_rmse},")
        else:
            f.write(f",,,")
    
        print(f"{msg} | Python MI runtime (s) | mean: {np.mean(runtimes)}, min: {np.min(runtimes)}, max: {np.max(runtimes)}")
        print(f"{msg} | Python MI y_hat - y mean | mean: {np.mean(biases)}, min: {np.min(biases)}, max: {np.max(biases)}")
        print(f"{msg} | Python MI y_hat - y SD | mean: {np.mean(sds)}, min: {np.min(sds)}, max: {np.max(sds)}")
        print(f"{msg} | Python MI # of discrepancies w.r.t. PyMICE per imputed dataset | mean: {np.mean(discrepancies_count)}, min: {np.min(discrepancies_count)}, max: {np.max(discrepancies_count)}")

        f.write(f"{np.mean(biases)},{np.mean(sds)},{np.mean(discrepancies_count)},")
    
        if imp_biases is not None:
            print(f"{msg} | Python MI imputation diff mean | mean: {np.mean(imp_biases)}, min: {np.min(imp_biases)}, max: {np.max(imp_biases)}")
            f.write(f"{np.mean(imp_biases)},")
        else:
            f.write(",")
        if imp_sds is not None:
            print(f"{msg} | Python MI imputation diff SD | mean: {np.mean(imp_sds)}, min: {np.min(imp_sds)}, max: {np.max(imp_sds)}")
            f.write(f"{np.mean(imp_sds)},")
        else:
            f.write(",")
        if imp_accs is not None:
            print(f"{msg} | Python MI imputation accuracy | mean: {np.mean(imp_accs)}, min: {np.min(imp_accs)}, max: {np.max(imp_accs)}")
            f.write(f"{np.mean(imp_accs)},")
        else:
            f.write(",")
        if imp_aucs is not None:
            print(f"{msg} | Python MI imputation AUC | mean: {np.mean(imp_aucs)}, min: {np.min(imp_aucs)}, max: {np.max(imp_aucs)}")
            f.write(f"{np.mean(imp_aucs)},")
        else:
            f.write(",")
    
        f.write(f"{np.mean(runtimes)}\n")

### Data prep

In [92]:
# Data generated by siMICE scripts (https://github.com/Luyaochen1/midn_gui)
midn_sc1 = load_or_generate_midn_data(midn_scenario_rows, midn_scenario_cols, "scenario_1", midn_miss_rows_count, scenarios_miss_col, labels_noise_scale)
midn_sc2 = load_or_generate_midn_data(midn_scenario_rows, midn_scenario_cols, "scenario_2", midn_miss_rows_count, scenarios_miss_col, labels_noise_scale)

In [93]:
# Scenarios per data generation instructions in https://www.nature.com/articles/s41467-020-19270-2 
paper_sc1 = load_or_generate_paper_data(paper_scenario_rows, "scenario_1", scenarios_miss_col, labels_noise_scale)
paper_sc2 = load_or_generate_paper_data(paper_scenario_rows, "scenario_2", scenarios_miss_col, labels_noise_scale)

In [94]:
if run_real_setup:
    real = load_real_data(real_setup_outlier_threshold)

### Imputation via regression analysis

In [95]:
if run_scenarios:
    print_stats(*benchmark_scenario(scenarios_runs_count, midn_sc1, LinearRegression(), mi_factor, scenarios_miss_col, miss_val, dump_data))
    print_stats(*benchmark_scenario(scenarios_runs_count, midn_sc2, LogisticRegression(), mi_factor, scenarios_miss_col, miss_val, dump_data))
    print_stats(*benchmark_scenario(scenarios_runs_count, paper_sc1, LinearRegression(), mi_factor, scenarios_miss_col, miss_val, dump_data))
    print_stats(*benchmark_scenario(scenarios_runs_count, paper_sc2, LogisticRegression(), mi_factor, scenarios_miss_col, miss_val, dump_data))

Evaluating midn_scenario_1: 100%|██████████| 10/10 [00:00<00:00, 26.08it/s]


midn_scenario_1 | Python MI coefs bias: 0.045739661497189665
midn_scenario_1 | Python MI coefs SD: 0.00048788383657291103
midn_scenario_1 | Python MI coefs rMSE: 0.04574226343891913
midn_scenario_1 | Python MI runtime (s) | mean: 0.03821649551391602, min: 0.03712177276611328, max: 0.041043758392333984
midn_scenario_1 | Python MI y_hat - y mean | mean: 0.04163983426028867, min: 0.04148483500676961, max: 0.04176054235445908
midn_scenario_1 | Python MI y_hat - y SD | mean: 0.034456734264173804, min: 0.034386793483413634, max: 0.0345202950210817
midn_scenario_1 | Python MI # of discrepancies w.r.t. PyMICE per imputed dataset | mean: 0.0, min: 0.0, max: 0.0
midn_scenario_1 | Python MI imputation diff mean | mean: 0.496476294035667, min: 0.49579309725906806, max: 0.4969699733071077
midn_scenario_1 | Python MI imputation diff SD | mean: 0.07300040923182874, min: 0.0724372776436671, max: 0.07348675385223226
midn_scenario_1 | Python MI imputation accuracy | mean: 0.0, min: 0.0, max: 0.0
midn_sc

Evaluating midn_scenario_2: 100%|██████████| 10/10 [00:00<00:00, 24.84it/s]


midn_scenario_2 | Python MI coefs bias: 0.03679470668483106
midn_scenario_2 | Python MI coefs SD: 4.002966042486721e-16
midn_scenario_2 | Python MI coefs rMSE: 0.03679470668483098
midn_scenario_2 | Python MI runtime (s) | mean: 0.04010117053985596, min: 0.03896331787109375, max: 0.0411219596862793
midn_scenario_2 | Python MI y_hat - y mean | mean: 0.03843508345521983, min: 0.03843508345521982, max: 0.03843508345521982
midn_scenario_2 | Python MI y_hat - y SD | mean: 0.031495021617380436, min: 0.031495021617380436, max: 0.031495021617380436
midn_scenario_2 | Python MI # of discrepancies w.r.t. PyMICE per imputed dataset | mean: 0.0, min: 0.0, max: 0.0
midn_scenario_2 | Python MI imputation diff mean | mean: 0.5199999999999999, min: 0.52, max: 0.52
midn_scenario_2 | Python MI imputation diff SD | mean: 0.4995998398718718, min: 0.4995998398718718, max: 0.4995998398718718
midn_scenario_2 | Python MI imputation accuracy | mean: 0.4800000000000001, min: 0.48, max: 0.48
midn_scenario_2 | Pyth

Evaluating paper_scenario_1: 100%|██████████| 10/10 [00:00<00:00, 60.90it/s]


paper_scenario_1 | Python MI coefs bias: 0.014595214051862248
paper_scenario_1 | Python MI coefs SD: 0.0005242313543084189
paper_scenario_1 | Python MI coefs rMSE: 0.014604625696419586
paper_scenario_1 | Python MI runtime (s) | mean: 0.016376352310180663, min: 0.016115188598632812, max: 0.016905546188354492
paper_scenario_1 | Python MI y_hat - y mean | mean: 0.028923728906859757, min: 0.028645548613483392, max: 0.02922581070600778
paper_scenario_1 | Python MI y_hat - y SD | mean: 0.01932852320117178, min: 0.019265083974741557, max: 0.01941441781346963
paper_scenario_1 | Python MI # of discrepancies w.r.t. PyMICE per imputed dataset | mean: 0.0, min: 0.0, max: 0.0
paper_scenario_1 | Python MI imputation diff mean | mean: 0.3928594638818056, min: 0.3925327099193102, max: 0.39302440706007674
paper_scenario_1 | Python MI imputation diff SD | mean: 0.23832416068284473, min: 0.23788606896622574, max: 0.2387872873893915
paper_scenario_1 | Python MI imputation accuracy | mean: 0.0, min: 0.0, m

Evaluating paper_scenario_2: 100%|██████████| 10/10 [00:00<00:00, 51.96it/s]

paper_scenario_2 | Python MI coefs bias: 0.26533890254348547
paper_scenario_2 | Python MI coefs SD: 1.1102230246251565e-16
paper_scenario_2 | Python MI coefs rMSE: 0.2653389025434856
paper_scenario_2 | Python MI runtime (s) | mean: 0.0192014217376709, min: 0.01887798309326172, max: 0.019766569137573242
paper_scenario_2 | Python MI y_hat - y mean | mean: 0.1073512439165559, min: 0.10735124391655591, max: 0.10735124391655591
paper_scenario_2 | Python MI y_hat - y SD | mean: 0.08681453021687595, min: 0.08681453021687596, max: 0.08681453021687596
paper_scenario_2 | Python MI # of discrepancies w.r.t. PyMICE per imputed dataset | mean: 0.0, min: 0.0, max: 0.0
paper_scenario_2 | Python MI imputation diff mean | mean: 0.5205992509363295, min: 0.5205992509363296, max: 0.5205992509363296
paper_scenario_2 | Python MI imputation diff SD | mean: 0.4995754906526761, min: 0.49957549065267615, max: 0.49957549065267615
paper_scenario_2 | Python MI imputation accuracy | mean: 0.4794007490636704, min: 0

### PyMICE

In [96]:
if run_pymice:
    print_stats(*benchmark_pymice(pymice_runs_count, midn_sc1[0], midn_sc1[1], midn_sc1[2], midn_sc1[3], mi_factor))
    print_stats(*benchmark_pymice(pymice_runs_count, midn_sc2[0], midn_sc2[1], midn_sc2[2], midn_sc2[3], mi_factor, binary=True))
    print_stats(*benchmark_pymice(pymice_runs_count, paper_sc1[0], paper_sc1[1], paper_sc1[2], paper_sc1[3], mi_factor))
    print_stats(*benchmark_pymice(pymice_runs_count, paper_sc2[0], paper_sc2[1], paper_sc2[2], paper_sc2[3], mi_factor, binary=True))

Evaluating pymice for midn_scenario_1: 100%|██████████| 10/10 [00:01<00:00,  8.51it/s]


midn_scenario_1 | Python MI coefs bias: 0.0421974123964411
midn_scenario_1 | Python MI coefs SD: 1.5700924586837752e-16
midn_scenario_1 | Python MI coefs rMSE: 0.04219741239644111
midn_scenario_1 | Python MI runtime (s) | mean: 0.11720175743103027, min: 0.11594915390014648, max: 0.11945986747741699
midn_scenario_1 | Python MI y_hat - y mean | mean: 0.040261327077052496, min: 0.040261327077052496, max: 0.040261327077052496
midn_scenario_1 | Python MI y_hat - y SD | mean: 0.03209697329865036, min: 0.03209697329865036, max: 0.03209697329865036
midn_scenario_1 | Python MI # of discrepancies w.r.t. PyMICE per imputed dataset | mean: -1.0, min: -1.0, max: -1.0
midn_scenario_1 | Python MI imputation diff mean | mean: 0.498542685889361, min: 0.49854268588936096, max: 0.49854268588936096
midn_scenario_1 | Python MI imputation diff SD | mean: 0.034444216138940845, min: 0.03444421613894085, max: 0.03444421613894085
midn_scenario_1 | Python MI imputation accuracy | mean: 0.0, min: 0.0, max: 0.0
mi

Evaluating pymice for midn_scenario_2: 100%|██████████| 10/10 [00:01<00:00,  8.39it/s]


midn_scenario_2 | Python MI coefs bias: 0.03563338030291722
midn_scenario_2 | Python MI coefs SD: 4.002966042486721e-16
midn_scenario_2 | Python MI coefs rMSE: 0.035633380302917385
midn_scenario_2 | Python MI runtime (s) | mean: 0.1188499927520752, min: 0.1170511245727539, max: 0.12030434608459473
midn_scenario_2 | Python MI y_hat - y mean | mean: 0.040495681414446864, min: 0.040495681414446864, max: 0.040495681414446864
midn_scenario_2 | Python MI y_hat - y SD | mean: 0.03179321979572112, min: 0.03179321979572111, max: 0.03179321979572111
midn_scenario_2 | Python MI # of discrepancies w.r.t. PyMICE per imputed dataset | mean: -1.0, min: -1.0, max: -1.0
midn_scenario_2 | Python MI imputation diff mean | mean: 0.54, min: 0.54, max: 0.54
midn_scenario_2 | Python MI imputation diff SD | mean: 0.4983974317750846, min: 0.49839743177508455, max: 0.49839743177508455
midn_scenario_2 | Python MI imputation accuracy | mean: 0.4600000000000001, min: 0.4600000000000001, max: 0.4600000000000001
mid

Evaluating pymice for paper_scenario_1: 100%|██████████| 10/10 [00:00<00:00, 35.76it/s]


paper_scenario_1 | Python MI coefs bias: 0.014137122713802899
paper_scenario_1 | Python MI coefs SD: 2.220446049250313e-16
paper_scenario_1 | Python MI coefs rMSE: 0.014137122713802863
paper_scenario_1 | Python MI runtime (s) | mean: 0.02788987159729004, min: 0.026172161102294922, max: 0.030284643173217773
paper_scenario_1 | Python MI y_hat - y mean | mean: 0.02825184900412257, min: 0.02825184900412257, max: 0.02825184900412257
paper_scenario_1 | Python MI y_hat - y SD | mean: 0.01881477450440707, min: 0.018814774504407068, max: 0.018814774504407068
paper_scenario_1 | Python MI # of discrepancies w.r.t. PyMICE per imputed dataset | mean: -1.0, min: -1.0, max: -1.0
paper_scenario_1 | Python MI imputation diff mean | mean: 0.3938982923802233, min: 0.3938982923802233, max: 0.3938982923802233
paper_scenario_1 | Python MI imputation diff SD | mean: 0.23576273568769485, min: 0.23576273568769482, max: 0.23576273568769482
paper_scenario_1 | Python MI imputation accuracy | mean: 0.0, min: 0.0, 

Evaluating pymice for paper_scenario_2: 100%|██████████| 10/10 [00:00<00:00, 37.45it/s]

paper_scenario_2 | Python MI coefs bias: 0.010157245324177992
paper_scenario_2 | Python MI coefs SD: 2.482534153247273e-16
paper_scenario_2 | Python MI coefs rMSE: 0.010157245324178103
paper_scenario_2 | Python MI runtime (s) | mean: 0.02662043571472168, min: 0.02616405487060547, max: 0.027194976806640625
paper_scenario_2 | Python MI y_hat - y mean | mean: 0.019777553379739493, min: 0.019777553379739493, max: 0.019777553379739493
paper_scenario_2 | Python MI y_hat - y SD | mean: 0.015762203890841382, min: 0.015762203890841386, max: 0.015762203890841386
paper_scenario_2 | Python MI # of discrepancies w.r.t. PyMICE per imputed dataset | mean: -1.0, min: -1.0, max: -1.0
paper_scenario_2 | Python MI imputation diff mean | mean: 0.36704119850187267, min: 0.36704119850187267, max: 0.36704119850187267
paper_scenario_2 | Python MI imputation diff SD | mean: 0.48199788080880773, min: 0.48199788080880773, max: 0.48199788080880773
paper_scenario_2 | Python MI imputation accuracy | mean: 0.6329588

### siMICE

In [97]:
if run_simice:
    print_stats("siMICE scenario 1", *benchmark_simice(simice_runs_count, "scenario_1", midn_sc1[1], midn_sc1[2], midn_sc1[3], mi_factor, dump_data))
    print_stats("siMICE scenario 2", *benchmark_simice(simice_runs_count, "scenario_2", midn_sc2[0], midn_sc2[2], midn_sc2[3], mi_factor, dump_data))

### Real data

In [98]:
if run_real_setup:
    print_stats("Real data", *benchmark_real_setup(real_setup_runs_count, real, mi_factor, miss_val, dump_data))
    print_stats("PyMICE real setup", *benchmark_pymice(real_setup_runs_count, real[0], real[1], real[3], real[2], mi_factor, non_miss_rows=real[-1]))